In [4]:
import torch
import os

dir_name = os.path.abspath(".")
model_name = "linear1"

PATH = dir_name + "\\" + model_name + ".pt"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.jit.load(PATH, map_location=device)
model.eval()

RecursiveScriptModule(
  original_name=LinearRegressionModel
  (linear): RecursiveScriptModule(original_name=Linear)
)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import sklearn.model_selection
from sklearn import preprocessing

df = pd.read_csv("..\medical_insurance.csv")
df.info()

x = df["bmi"].to_numpy()
y = df["charges"].to_numpy()

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.3, random_state=42)

x_test = x_test.astype(np.float32)
x_test = x_test.reshape(-1, 1)

y_test = y_test.astype(np.float32)
y_test = y_test.reshape(-1, 1)

data_normalizer = preprocessing.MinMaxScaler()

x_test = data_normalizer.fit_transform(x_test)
y_test = data_normalizer.fit_transform(y_test)
y_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2772 entries, 0 to 2771
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       2772 non-null   int64  
 1   sex       2772 non-null   object 
 2   bmi       2772 non-null   float64
 3   children  2772 non-null   int64  
 4   smoker    2772 non-null   object 
 5   region    2772 non-null   object 
 6   charges   2772 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 151.7+ KB


array([[1.25562102e-01],
       [4.30647761e-01],
       [1.74153298e-01],
       [8.95029493e-03],
       [3.62575389e-02],
       [3.66888553e-01],
       [5.78540117e-02],
       [7.36616790e-01],
       [1.18678443e-01],
       [2.42171437e-02],
       [1.66500717e-01],
       [1.51198238e-01],
       [2.19724178e-02],
       [3.15360993e-01],
       [5.13749570e-02],
       [9.06259343e-02],
       [4.65497851e-01],
       [1.39494866e-01],
       [4.21305485e-02],
       [3.66134614e-01],
       [1.47633702e-01],
       [9.17007506e-01],
       [2.94866115e-01],
       [7.61431336e-01],
       [5.23326814e-01],
       [1.18531339e-01],
       [7.97251053e-03],
       [2.08921999e-01],
       [3.74807924e-01],
       [0.00000000e+00],
       [1.50966555e-01],
       [5.74048311e-02],
       [8.26338492e-03],
       [9.55297053e-03],
       [4.33625698e-01],
       [1.11796834e-01],
       [8.66940431e-03],
       [2.03392059e-02],
       [8.08359087e-02],
       [3.21436048e-01],


In [5]:
# inference model


RecursiveScriptModule(
  original_name=LinearRegressionModel
  (linear): RecursiveScriptModule(original_name=Linear)
)